In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Preparacion de la exploracion

___PATH CASA___

In [4]:
test_path = '/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/lichess_db_standard_rated_2020-02.pgn.bz2'

___PATH TRABAJO___

In [3]:
test_path = '/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/lichess_db_standard_rated_2020-02.pgn.bz2'

__CODIGO PARA GENERAR SAMPLE__

In [5]:
!bzcat {test_path} | head -n 2000 > games_sample.csv

"bzcat" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
sample_path = '/home/dsc/Data_Science_Projects/Chess_Engine_TFM/data/games_sample.csv'
sample = pd.read_csv(sample_path, sep=';', header=None)

# TRABAJO EN CURSO

In [2]:
sample_path = 'data/games_sample.csv'
sample = pd.read_csv(sample_path, sep=';', header=None)

In [196]:
def read_games(sample_df, games, eval='yes', process_games='process'):
    cols = [
        'Event', 
        'Site', 
        'Date', 
        'Round', 
        'WhiteUser', 
        'BlackUser', 
        'Result', 
        'UTCDate', 
        'UTCTime', 
        'WHiteElo', 
        'BlackElo', 
        'WhiteRatingDiff', 
        'BlackRatingDiff', 
        'ECO', 
        'Opening', 
        'TimeControl', 
        'Termination', 
        'Game'
        ]
    df = pd.DataFrame(columns=cols)
    game_row = 0
    for i in range(0, games):
        game = sample_df.iloc[game_row:game_row+18].transpose()
        game.columns = cols
        df = pd.concat([df, game])
        game_row += 18
    df.reset_index(drop=True, inplace=True)
    

    # Game processing
    if process_games == 'process':
            # Clean all columns but GAME, which needs to remain raw
            df.loc[:, df.columns != 'Game'] = df.loc[:, df.columns != 'Game'].applymap(lambda x: x.split(' ', 1)[1].replace('"', '')[:-1])
            
            # Format date column
            df['Date'] = pd.to_datetime(df['UTCDate'])

            # Find game type by time control and set a variable for it
            game_type = pd.DataFrame(columns=['Bullet', 'Blitz', 'Rapid', 'Classical'])
            for i in df['Event']:
                bullet = ((i.find('Bullet')>0)*1)
                blitz = ((i.find('Blitz')>0)*1)
                rapid = ((i.find('Rapid')>0)*1)
                classical = ((i.find('Classical')>0)*1)
                gamedf = pd.DataFrame([{'Bullet':bullet,
                                        'Blitz':blitz,
                                        'Rapid':rapid,
                                        'Classical':classical}])
                game_type = pd.concat([game_type, gamedf], ignore_index=True)
            df = df.join(game_type)
            
            # Final Step: Drop unuseful columns
            df.drop(['Event', 'Site', 'ECO', 'BlackRatingDiff', 'WhiteUser', 'BlackUser', 'UTCDate', 'Round'], axis=1, inplace=True)
    # Gets only evaluated games

    if eval == 'yes':
        df = df[df['Game'].apply(lambda x: x.find('eval')>0)==True]


    return df

In [199]:
test = read_games(sample, 100)

In [ ]:
test.to_csv('/data/games_processed_sample.csv')

In [ ]:
eval_games = pd.DataFrame()
for chunk in fulldf:
    chunkdf = read_games(chunk, len(chunk)/18)
    eval_games = pd.concat([eval_games, chunkdf])
